In [1]:
!pip install -U easynmt
!pip install sacremoses
!pip install simpletransformers
!pip install nltk
!pip install torch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19900 sha256=b952101fc15c4452ddeeec1270c64dc2680af4893c7cd03c9ca6ea91fa57249b
  Stored in directory: /root/.cache/pip/wheels/ab/55/72/aba4face7eac1d7750ca700aa1797b135fb8915e949da504cc
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=0e446e2c44b82f36f221108458f7df70f7aa1a38ab3f73dfb2494a97006c8708
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built easynmt fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
import torch
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import random
from easynmt import EasyNMT
import nltk


from transformers import Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
from transformers import BertPreTrainedModel, BertModel

import pandas as pd
import numpy as np
import os
from pathlib import Path

from sklearn.metrics import classification_report

if not torch.cuda.is_available():
    if torch.backends.mps.is_available():
        DEVICE = 'mps'
    else:
        DEVICE = 'cpu'
else:
    DEVICE = 'cuda:0'
print("Device:", DEVICE)
device = torch.device(DEVICE)

Device: cuda:0


In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Now try downloading the punkt data again
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Augment dataset or load csv if already available

In [5]:
def augment_text(df, languages, num_samples_needed, batch_size=128):
    augmented_texts = []
    total_augmented = 0
    model = EasyNMT('opus-mt')

    while total_augmented < num_samples_needed:
        # Process texts in batches
        for i in range(0, len(df), batch_size):
            # Process texts in batches
            print(f"Processing batch {i} to {i + batch_size}...")
            batch_df = df.iloc[i:i + batch_size]  # Get a batch of rows
            batch_texts = batch_df['text'].tolist()
            if total_augmented >= num_samples_needed:
                break  # Stop if we already have enough samples

            # Translate each batch
            lang = random.choice(languages)
            translated_texts = model.translate(batch_texts, source_lang='en', target_lang=lang, batch_size=batch_size)
            back_translated_texts = model.translate(translated_texts, source_lang=lang, target_lang='en', batch_size=batch_size)

            # Create new rows with the augmented texts and all other original data
            for index, (original, back_translated) in enumerate(zip(batch_texts, back_translated_texts)):
                new_row = batch_df.iloc[index].to_dict()  # Convert the original row to a dictionary
                new_row['text'] = back_translated  # Replace the original text with the augmented one
                augmented_texts.append(new_row)
                total_augmented += 1
                if total_augmented % 100 == 0:
                    print(f"Augmented {total_augmented} samples out of {num_samples_needed} needed.")
                if total_augmented >= num_samples_needed:
                    break  # Stop if we already have enough samples

            if total_augmented >= num_samples_needed:
                break  # Stop if we already have enough samples

    return pd.DataFrame(augmented_texts)


In [4]:
from dont_patronize_me import DontPatronizeMe
import pandas as pd

def load_datasets(languages=None):
    dpm = DontPatronizeMe('.', '.')
    dpm.load_task1()
    trids = pd.read_csv('internal_train_par_ids.csv')
    teids = pd.read_csv('internal_dev_par_ids.csv')

    trids.par_id = trids.par_id.astype(str)
    teids.par_id = teids.par_id.astype(str)

    data=dpm.train_task1_df

    rows = [] # will contain par_id, label and text
    for idx in range(len(trids)):
        parid = trids.par_id[idx]
        keyword = data.loc[data.par_id == parid].keyword.values[0]
        text = data.loc[data.par_id == parid].text.values[0]
        orig_label = int(data.loc[data.par_id == parid].orig_label.values[0])
        label = data.loc[data.par_id == parid].label.values[0]
        rows.append({
            'par_id':parid,
            'community':keyword,
            'text':text,
            'label':label,
            'orig_label':orig_label
        })

    trdf1 = pd.DataFrame(rows)

    if languages is not None:
        traindf_majority = trdf1[trdf1['label'] == 0]
        traindf_minority = trdf1[trdf1['label'] == 1]
        # Calculate the number of samples needed to match the majority class
        num_samples_needed = len(traindf_majority) - len(traindf_minority)
        traindf_minority_augmented = augment_text(traindf_minority, languages, num_samples_needed)
        traindf_combined = pd.concat([traindf_majority, traindf_minority, traindf_minority_augmented])
        traindf_combined = traindf_combined.sample(frac=1, random_state=42).reset_index(drop=True)
        trdf1 = traindf_combined

    rows = [] # will contain par_id, label and text
    for idx in range(len(teids)):
        parid = teids.par_id[idx]
        #print(parid)
        # select row from original dataset
        keyword = data.loc[data.par_id == parid].keyword.values[0]
        text = data.loc[data.par_id == parid].text.values[0]
        orig_label = int(data.loc[data.par_id == parid].orig_label.values[0])
        label = data.loc[data.par_id == parid].label.values[0]
        rows.append({
            'par_id':parid,
            'community':keyword,
            'text':text,
            'label':label,
            'orig_label':orig_label
        })

    tedf1 = pd.DataFrame(rows)

    if languages is not None:
        return trdf1, tedf1, traindf_minority_augmented
    else:
        return trdf1, tedf1

In [ ]:
trdf1, devdf, augmented_df = load_datasets(languages=['de'])

11.9kB [00:00, 12.5MB/s]                   

Processing batch 0 to 128...


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Augmented 100 samples out of 5091 needed.
Processing batch 128 to 256...
Augmented 200 samples out of 5091 needed.
Processing batch 256 to 384...
Augmented 300 samples out of 5091 needed.
Processing batch 384 to 512...
Augmented 400 samples out of 5091 needed.
Augmented 500 samples out of 5091 needed.
Processing batch 512 to 640...
Processing batch 0 to 128...
Augmented 600 samples out of 5091 needed.
Augmented 700 samples out of 5091 needed.
Processing batch 128 to 256...
Augmented 800 samples out of 5091 needed.
Processing batch 256 to 384...
Augmented 900 samples out of 5091 needed.
Processing batch 384 to 512...
Augmented 1000 samples out of 5091 needed.
Augmented 1100 samples out of 5091 needed.
Processing batch 512 to 640...
Processing batch 0 to 128...
Augmented 1200 samples out of 5091 needed.
Augmented 1300 samples out of 5091 needed.
Processing batch 128 to 256...
Augmented 1400 samples out of 5091 needed.
Processing batch 256 to 384...
Augmented 1500 samples out of 5091 need

In [ ]:
trdf1

,par_id,community,text,label,orig_label
0,6477,refugee,Tensions remain high at Australia 's island re...,0,0
1,4850,poor-families,"""Picariello , also known as """" Emperor Pic , ""...",0,0
2,2473,refugee,"""Irungu , who also serves as the head of the N...",0,0
3,5344,disabled,"Mrs. Aviva Dankner , owner of Castra Mall , al...",0,0
4,7070,hopeless,2 : The ' Check Engine ' and ' ABS ' lights ca...,0,0
...,...,...,...,...,...
5086,8081,poor-families,"""-- TEFAP : The emergency food assistance prog...",1,3
5087,7846,in-need,"""Filmstar Reema also believes that this isn't ...",1,4
5088,5432,vulnerable,"""A large-scale publicly funded energy efficien...",1,3
5089,10333,hopeless,"""""For the children, it shows that someone love...",1,4


In [ ]:
# Save to CSV
trdf1.to_csv('german_augmented_train_set.csv', index=False)

## French

In [ ]:
trdf1, devdf, augmented_df = load_datasets(languages=['fr'])

11.9kB [00:00, 16.2MB/s]                   

Processing batch 0 to 128...


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Augmented 100 samples out of 5091 needed.
Processing batch 128 to 256...
Augmented 200 samples out of 5091 needed.
Processing batch 256 to 384...
Augmented 300 samples out of 5091 needed.
Processing batch 384 to 512...
Augmented 400 samples out of 5091 needed.
Augmented 500 samples out of 5091 needed.
Processing batch 512 to 640...
Processing batch 0 to 128...
Augmented 600 samples out of 5091 needed.
Augmented 700 samples out of 5091 needed.
Processing batch 128 to 256...
Augmented 800 samples out of 5091 needed.
Processing batch 256 to 384...
Augmented 900 samples out of 5091 needed.
Processing batch 384 to 512...
Augmented 1000 samples out of 5091 needed.
Augmented 1100 samples out of 5091 needed.
Processing batch 512 to 640...
Processing batch 0 to 128...
Augmented 1200 samples out of 5091 needed.
Augmented 1300 samples out of 5091 needed.
Processing batch 128 to 256...
Augmented 1400 samples out of 5091 needed.
Processing batch 256 to 384...
Augmented 1500 samples out of 5091 need

In [ ]:
trdf1

,par_id,community,text,label,orig_label
0,6477,refugee,Tensions remain high at Australia 's island re...,0,0
1,4850,poor-families,"""Picariello , also known as """" Emperor Pic , ""...",0,0
2,2473,refugee,"""Irungu , who also serves as the head of the N...",0,0
3,5344,disabled,"Mrs. Aviva Dankner , owner of Castra Mall , al...",0,0
4,7070,hopeless,2 : The ' Check Engine ' and ' ABS ' lights ca...,0,0
...,...,...,...,...,...
5086,8081,poor-families,"""--- TEFAP: The Emergency Food Aid Program, it...",1,3
5087,7846,in-need,"""The movie star Reema also thinks that this is...",1,4
5088,5432,vulnerable,There is an urgent need for a state-funded lar...,1,3
5089,10333,hopeless,"""""""For the children, he shows that someone lov...",1,4


In [ ]:
trdf1.to_csv('french_augmented_train_set.csv', index=False)

## Spanish

In [ ]:
trdf1, devdf, augmented_df = load_datasets(languages=['es'])

Processing batch 0 to 128...


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
trdf1

In [ ]:
trdf1.to_csv('spanish_augmented_train_set.csv', index=False)

## Chinese

In [ ]:
trdf1, devdf, augmented_df = load_datasets(languages=['zh'])

Processing batch 0 to 128...


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
trdf1

In [ ]:
trdf1.to_csv('chinese_augmented_train_set.csv', index=False)

## Italian

In [ ]:
trdf1, devdf, augmented_df = load_datasets(languages=['it'])

Processing batch 0 to 128...


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
trdf1

In [ ]:
trdf1.to_csv('italian_augmented_train_set.csv', index=False)

## Mixed

In [ ]:
trdf1, devdf, augmented_df = load_datasets(languages=['de', 'fr', 'es', 'zh', 'it'])

Processing batch 0 to 32...
Processing batch 32 to 64...
Processing batch 64 to 96...
Processing batch 96 to 128...
Augmented 100 samples out of 5091 needed.
Processing batch 128 to 160...
Processing batch 160 to 192...
Processing batch 192 to 224...
Augmented 200 samples out of 5091 needed.
Processing batch 224 to 256...
Processing batch 256 to 288...
Processing batch 288 to 320...
Augmented 300 samples out of 5091 needed.
Processing batch 320 to 352...
Processing batch 352 to 384...
Processing batch 384 to 416...
Augmented 400 samples out of 5091 needed.
Processing batch 416 to 448...
Processing batch 448 to 480...
Processing batch 480 to 512...
Augmented 500 samples out of 5091 needed.
Processing batch 512 to 544...
Processing batch 544 to 576...
Processing batch 576 to 608...
Processing batch 0 to 32...
Augmented 600 samples out of 5091 needed.
Processing batch 32 to 64...
Processing batch 64 to 96...
Processing batch 96 to 128...
Augmented 700 samples out of 5091 needed.
Processin

In [ ]:
trdf1

,par_id,community,text,label,orig_label
0,4567,refugee,"""Brussels - Top European Union official Donald...",0,0
1,9081,hopeless,"""""She was in a vulnerable, powerless, desperat...",1,3
2,9690,women,"And I found that, especially for one of my wom...",1,4
3,7015,refugee,"""We told the president of Maldives that we can...",1,4
4,4754,women,NAN also reports that the initiative provides ...,0,0
...,...,...,...,...,...
11367,6055,refugee,"All this seems to be very theoretical, but in ...",1,3
11368,1810,immigrant,"""Castillo is thrilled that her novel -- popula...",0,0
11369,2965,migrant,From refugee to runway : How migrants are tran...,0,0
11370,1594,migrant,"""During a fact-finding visit to Algeciras on S...",0,0


In [ ]:
trdf1.to_csv('mixed_augmented_train_set.csv', index=False)

# Evaluating

In [6]:
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

cuda_available=torch.cuda.is_available()

## German

In [7]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('german_augmented_train_set.csv')

In [8]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Running Epoch 4 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.562874251497006
recall：  0.4723618090452261
f1_score:  0.5136612021857923


## French

In [9]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('french_augmented_train_set.csv')

In [10]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True,
                                      )
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.5973154362416108
recall：  0.4472361809045226
f1_score:  0.5114942528735633


## Spanish

In [11]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('spanish_augmented_train_set.csv')

In [12]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.5696969696969697
recall：  0.4723618090452261
f1_score:  0.5164835164835164


## Chinese

In [13]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('chinese_augmented_train_set.csv')

In [14]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.5212765957446809
recall：  0.49246231155778897
f1_score:  0.5064599483204134


## Italian

In [15]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('italian_augmented_train_set.csv')

In [16]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.5340909090909091
recall：  0.4723618090452261
f1_score:  0.5013333333333333


## Mixed

In [17]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('mixed_augmented_train_set.csv')

In [18]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=5,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.5130890052356021
recall：  0.49246231155778897
f1_score:  0.5025641025641026


## Mixed 2 epochs

In [19]:
trdf1, devdf = load_datasets()
trdf1 = pd.read_csv('mixed_augmented_train_set.csv')

In [20]:
torch.manual_seed(42)
task1_model_args = ClassificationArgs(num_train_epochs=2,
                                      no_save=True,
                                      no_cache=True,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta",
                                  'roberta-base',
                                  args = task1_model_args,
                                  num_labels=2,
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(trdf1[['text', 'label']])
# run predictions on dev
preds_task1, _ = task1_model.predict(devdf.text.tolist())
print("precision: ", precision_score(devdf.label.values, preds_task1))
print("recall： ", recall_score(devdf.label.values, preds_task1))
print("f1_score: ", f1_score(devdf.label.values, preds_task1))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1422 [00:00<?, ?it/s]

Running Epoch 2 of 2:   0%|          | 0/1422 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

precision:  0.5625
recall：  0.49748743718592964
f1_score:  0.5279999999999999
